In [60]:
import polars as pl
from pathlib import Path
import datetime

In [61]:
files_dir = Path("../files")  # Dossier au même niveau que le répertoire actuel

In [62]:
# Charger le CSV avec les bonnes colonnes
df = pl.read_csv(
    files_dir/ "nom_prenoms_lower_trim.csv",
    separator=",",  # Ajuster selon le séparateur réel
    new_columns=["nom", "prenoms", "sexe", "age"],
    dtypes={
        "nom": pl.Utf8,
        "prenoms": pl.Utf8,
        "sexe": pl.Categorical,  # Optimisation mémoire
        "age": pl.UInt8
    }
)

C:\Users\USER\AppData\Local\Temp\ipykernel_18788\1450877043.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(


In [63]:
df

nom,prenoms,sexe,age
str,str,cat,u8
"""lago""","""serge""","""Masculin""",43
"""lago""","""renaud""","""Masculin""",40
"""lago""","""therese""","""Féminin""",68
"""bode""","""charleine""","""Féminin""",25
"""dina""","""dina""","""Féminin""",27
…,…,…,…
"""diallo""","""maiga""","""Masculin""",46
"""diallo""","""amina""","""Féminin""",28
"""diallo""","""ramata""","""Féminin""",21


In [64]:
import polars as pl
from pathlib import Path

files_dir = Path("../files")

In [65]:
df = pl.read_csv(
    files_dir / "nom_prenoms_lower_trim.csv",
    new_columns=["nom", "prenoms", "sexe", "age"],
    dtypes={
        "nom": pl.Utf8,
        "prenoms": pl.Utf8,
        "sexe": pl.Categorical,
        "age": pl.UInt8
    }
)


C:\Users\USER\AppData\Local\Temp\ipykernel_18788\3926850597.py:1: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(


In [66]:
current_year = 2021
df = df.with_columns(annee_naissance=current_year - pl.col("age"))


In [67]:
df_exploded = df.with_columns(
    pl.col("prenoms").str.split(" ").alias("prenoms_split")
).explode("prenoms_split").rename({"prenoms_split": "prenom_explode"})


In [68]:
names_years = pl.concat([
    df_exploded.select("prenom_explode", "annee_naissance").rename({"prenom_explode": "name"}),
    df.select("nom", "annee_naissance").rename({"nom": "name"})
])

# Calcul de la plage temporelle
min_year = names_years["annee_naissance"].min()
max_year = current_year


In [69]:
df_exploded

nom,prenoms,sexe,age,annee_naissance,prenom_explode
str,str,cat,u8,u16,str
"""lago""","""serge""","""Masculin""",43,1978,"""serge"""
"""lago""","""renaud""","""Masculin""",40,1981,"""renaud"""
"""lago""","""therese""","""Féminin""",68,1953,"""therese"""
"""bode""","""charleine""","""Féminin""",25,1996,"""charleine"""
"""dina""","""dina""","""Féminin""",27,1994,"""dina"""
…,…,…,…,…,…
"""diallo""","""maiga""","""Masculin""",46,1975,"""maiga"""
"""diallo""","""amina""","""Féminin""",28,1993,"""amina"""
"""diallo""","""ramata""","""Féminin""",21,2000,"""ramata"""


In [70]:
# %% [markdown]
# ## 6. Statistiques annuelles (version compatible) - VERSION 1

# %%
# Création du pivot temporel
yearly_stats = (
    names_years
    .group_by("name", "annee_naissance")
    .agg(pl.count().alias("count"))
    .pivot(
        values="count",
        index="name",
        columns="annee_naissance",
        aggregate_function="sum"
    )
    .fill_null(0)
)

# Renommage manuel des colonnes
yearly_stats = yearly_stats.rename({
    str(col): f"annee_{col}" 
    for col in yearly_stats.columns 
    if col != "name"
})

# Gestion des années manquantes
min_year = names_years["annee_naissance"].min()
max_year = current_year
all_years = [f"annee_{y}" for y in range(min_year, max_year + 1)]

# Ajout des colonnes manquantes
for y_col in all_years:
    if y_col not in yearly_stats.columns:
        yearly_stats = yearly_stats.with_columns(pl.lit(0).alias(y_col))

# Tri des colonnes
yearly_stats = yearly_stats.select(["name"] + sorted(
    [col for col in yearly_stats.columns if col != "name"],
    key=lambda x: int(x.split("_")[1])
))

C:\Users\USER\AppData\Local\Temp\ipykernel_18788\42145864.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("count"))
C:\Users\USER\AppData\Local\Temp\ipykernel_18788\42145864.py:10: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


In [71]:
yearly_stats

name,annee_1886,annee_1887,annee_1888,annee_1889,annee_1890,annee_1891,annee_1892,annee_1893,annee_1894,annee_1895,annee_1896,annee_1897,annee_1898,annee_1899,annee_1900,annee_1901,annee_1902,annee_1903,annee_1904,annee_1905,annee_1906,annee_1907,annee_1908,annee_1909,annee_1910,annee_1911,annee_1912,annee_1913,annee_1914,annee_1915,annee_1916,annee_1917,annee_1918,annee_1919,annee_1920,annee_1921,…,annee_1985,annee_1986,annee_1987,annee_1988,annee_1989,annee_1990,annee_1991,annee_1992,annee_1993,annee_1994,annee_1995,annee_1996,annee_1997,annee_1998,annee_1999,annee_2000,annee_2001,annee_2002,annee_2003,annee_2004,annee_2005,annee_2006,annee_2007,annee_2008,annee_2009,annee_2010,annee_2011,annee_2012,annee_2013,annee_2014,annee_2015,annee_2016,annee_2017,annee_2018,annee_2019,annee_2020,annee_2021
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""abdul-aziz""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1,0,0,1,0,0,1,0,2,0,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,0,3,3,1,3,1,2,4
"""lantoum""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
"""grand""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,2,4,2,1,5,6,4,3,3,4,8,8,6,5,3,6,10,4,11,9,6,7,6,9,6,8,7,10,5,8,9,16,5,13,6,10,9
"""julienne""",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,…,111,150,123,101,151,155,146,91,123,98,120,151,103,120,121,126,127,125,120,102,99,110,105,89,110,106,103,99,119,89,93,121,96,111,93,63,76
"""ehnoc""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""weke douwa""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""djiobe""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
"""gregroird""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
stats = (
    pl.concat([
        df_exploded.select(
            pl.col("prenom_explode").alias("nom"), 
            "sexe"
        ).with_columns(type=pl.lit("prenoms")),
        
        df.select(
            pl.col("nom").alias("nom"),
            "sexe"
        ).with_columns(type=pl.lit("famille"))
    ])
    .group_by("nom", "sexe", "type")
    .agg(count=pl.len())
    .pivot(
        values="count",
        index="nom",
        columns=["sexe", "type"],
        aggregate_function="sum"
    )
    .fill_null(0)
    .rename({
        '{"Féminin","famille"}': "nombre_femme_nom_famille",
        '{"Féminin","prenoms"}': "nombre_femme_prenom",
        '{"Masculin","famille"}': "nombre_homme_nom_famille",
        '{"Masculin","prenoms"}': "nombre_homme_prenom"  # Correction orthographique
    })
    
)


C:\Users\USER\AppData\Local\Temp\ipykernel_18788\2881166339.py:15: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


In [73]:
stats

nom,nombre_femme_prenom,nombre_homme_prenom,nombre_homme_nom_famille,nombre_femme_nom_famille
str,u32,u32,u32,u32
"""kinedenin""",5,0,0,0
"""bouliguissa""",1,0,0,0
"""sousunle""",0,1,0,0
"""naise""",9,2,1,0
"""tedjouma""",4,0,0,0
…,…,…,…,…
"""koualerou""",0,0,0,1
"""moyemi""",2,0,0,0
"""huberdine""",1,0,0,0


In [74]:
prenom_unique_stats = (
    df_exploded.filter(pl.col("prenoms") == pl.col("prenom_explode"))
    .group_by("prenom_explode")
    .agg(prenom_unique=pl.len())
)

In [75]:
final_stats = (
    stats.join(
        prenom_unique_stats,
        left_on="nom",
        right_on="prenom_explode",
        how="left"
    )
    .with_columns(
        partie_prenom=pl.col("nombre_homme_prenom") + pl.col("nombre_femme_prenom"),
        nom_famille=pl.col("nombre_homme_nom_famille") + pl.col("nombre_femme_nom_famille"),
        prenom_unique=pl.coalesce("prenom_unique", 0)
    )
    .select([
        "nom",
        "partie_prenom",
        "nom_famille",
        "prenom_unique",
        "nombre_homme_prenom",
        "nombre_homme_nom_famille",
        "nombre_femme_prenom",
        "nombre_femme_nom_famille"
    ])
)

In [76]:
final_stats

nom,partie_prenom,nom_famille,prenom_unique,nombre_homme_prenom,nombre_homme_nom_famille,nombre_femme_prenom,nombre_femme_nom_famille
str,u32,u32,u32,u32,u32,u32,u32
"""kinedenin""",5,0,5,0,0,5,0
"""bouliguissa""",1,0,1,0,0,1,0
"""sousunle""",1,0,0,1,0,0,0
"""naise""",11,1,2,2,1,9,0
"""tedjouma""",4,0,4,0,0,4,0
…,…,…,…,…,…,…,…
"""koualerou""",0,1,0,0,0,0,1
"""moyemi""",2,0,1,0,0,2,0
"""huberdine""",1,0,0,0,0,1,0


In [77]:
final_stats = (
    final_stats
    .join(
        yearly_stats.unique(subset=["name"]),  # Élimination des doublons
        left_on="nom",
        right_on="name",
        how="left"
    )
    .pipe(lambda df: df.with_columns(
        partie_prenom=pl.coalesce("nombre_homme_prenom", 0) + pl.coalesce("nombre_femme_prenom", 0),
        nom_famille=pl.coalesce("nombre_homme_nom_famille", 0) + pl.coalesce("nombre_femme_nom_famille", 0)
    ))
)

In [78]:
final_stats

nom,partie_prenom,nom_famille,prenom_unique,nombre_homme_prenom,nombre_homme_nom_famille,nombre_femme_prenom,nombre_femme_nom_famille,annee_1886,annee_1887,annee_1888,annee_1889,annee_1890,annee_1891,annee_1892,annee_1893,annee_1894,annee_1895,annee_1896,annee_1897,annee_1898,annee_1899,annee_1900,annee_1901,annee_1902,annee_1903,annee_1904,annee_1905,annee_1906,annee_1907,annee_1908,annee_1909,annee_1910,annee_1911,annee_1912,annee_1913,annee_1914,…,annee_1985,annee_1986,annee_1987,annee_1988,annee_1989,annee_1990,annee_1991,annee_1992,annee_1993,annee_1994,annee_1995,annee_1996,annee_1997,annee_1998,annee_1999,annee_2000,annee_2001,annee_2002,annee_2003,annee_2004,annee_2005,annee_2006,annee_2007,annee_2008,annee_2009,annee_2010,annee_2011,annee_2012,annee_2013,annee_2014,annee_2015,annee_2016,annee_2017,annee_2018,annee_2019,annee_2020,annee_2021
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""kinedenin""",5,0,5,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
"""bouliguissa""",1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""sousunle""",1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""naise""",11,1,2,2,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,2,1,0
"""tedjouma""",4,0,4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""koualerou""",0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""moyemi""",2,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
"""huberdine""",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [79]:
final_stats.write_parquet(
     files_dir / "nom_prenoms_overlay_vf_lower_trim.parquet",
       compression="zstd"
)